<a href="https://colab.research.google.com/github/PavankumarUppar/DLL/blob/main/Program4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Splitting training dataset into train and validation sets
train_data, val_data = train_test_split(train_dataset, test_size=0.2, random_state=42, shuffle=True)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Create an instance of the CNN
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}")

# Evaluation on validation set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total}%")

# Evaluation on test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total}%")


Epoch [1/5], Step [100/750], Loss: 1.6203
Epoch [1/5], Step [200/750], Loss: 1.6402
Epoch [1/5], Step [300/750], Loss: 1.6096
Epoch [1/5], Step [400/750], Loss: 1.4902
Epoch [1/5], Step [500/750], Loss: 1.5550
Epoch [1/5], Step [600/750], Loss: 1.5586
Epoch [1/5], Step [700/750], Loss: 1.5635
Epoch [2/5], Step [100/750], Loss: 1.5782
Epoch [2/5], Step [200/750], Loss: 1.6024
Epoch [2/5], Step [300/750], Loss: 1.5735
Epoch [2/5], Step [400/750], Loss: 1.6301
Epoch [2/5], Step [500/750], Loss: 1.4964
Epoch [2/5], Step [600/750], Loss: 1.4797
Epoch [2/5], Step [700/750], Loss: 1.5157
Epoch [3/5], Step [100/750], Loss: 1.4655
Epoch [3/5], Step [200/750], Loss: 1.4761
Epoch [3/5], Step [300/750], Loss: 1.4924
Epoch [3/5], Step [400/750], Loss: 1.4704
Epoch [3/5], Step [500/750], Loss: 1.4613
Epoch [3/5], Step [600/750], Loss: 1.4612
Epoch [3/5], Step [700/750], Loss: 1.4693
Epoch [4/5], Step [100/750], Loss: 1.5047
Epoch [4/5], Step [200/750], Loss: 1.4942
Epoch [4/5], Step [300/750], Loss: